In [2]:
import pandas as pd
import geopandas as gpd

from dash import Dash, html, Output, Input,dash_table
import dash_bootstrap_components as dbc

import dash_leaflet as dl
import dash_leaflet.express as dlx
from dash_extensions.javascript import arrow_function, assign
import plotly.figure_factory as ff

import json

from matplotlib import cm, colors

In [3]:
oct_speed = pd.read_csv('static_data/raw_oct_speed.csv')
nov_speed = pd.read_csv('static_data/raw_nov_speed.csv')

In [74]:
ntas = gpd.read_file("shapefiles/nynta2020_24d")
df = pd.read_csv('static_data/rolling_avg.csv')

In [6]:
table_data = pd.read_csv('static_data/borough_speeds.csv')

In [75]:
def produce_rolling(df,window_size,min):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['NTAName', 'date'])
    df['avg_imputed'] = df.groupby('NTAName')['avg'].transform(lambda x: x.fillna(x.mean()))
    df['rolling_avg'] = df.groupby('NTAName')['avg_imputed'].rolling(window=window_size, min_periods=min).mean().reset_index(level=0, drop=True)
    return df

In [8]:
def make_table(df):
    boroughs = df.groupby('borough')['average_road_speed'].mean().reset_index(name='avg_speed')
    whole_city = df['average_road_speed'].mean()
    table = pd.DataFrame({'borough': 'NYC Whole', 'avg_speed': whole_city}, index=[0])
    boroughs = pd.concat([boroughs, table], ignore_index=True)
    boroughs['avg_speed'] = boroughs['avg_speed'].round(1)
    return boroughs

In [76]:
rolling_df = produce_rolling(df,3,1)
most_recent_df = rolling_df.loc[rolling_df.groupby('NTAName')['date'].idxmax()]
rolling_map = ntas.merge(most_recent_df, left_on='NTAName', right_on='NTAName')

In [67]:
scrape_date = rolling_df['date'].max().strftime('on %m/%d/%Y at %H:%M')

In [72]:
# make a datetime
rolling_df['date'] = pd.to_datetime(rolling_df['date'])
scrape_date = rolling_df['date'].max().strftime('on %m/%d/%Y at %H:%M')

In [73]:
scrape_date

'on 12/30/2024 at 13:24'

In [77]:
rolling_map = rolling_map[['NTAName','rolling_avg','Shape_Leng','Shape_Area','geometry']]

In [24]:
month_dict = {
    1 : 'January',
    2 : 'February',
    3 : 'March',
    4 : 'April',
    5 : 'May',
    6 : 'June',
    7 : 'July',
    8 : 'August',
    9 : 'September',
    10 : 'October',
    11 : 'November',
    12 : 'December'
}

In [78]:
# change crs to 4326
rolling_map = rolling_map.to_crs(epsg=4326)

In [90]:
rolling_map.shape

(191, 5)

In [79]:
rolling_map.dropna(subset=['rolling_avg'],inplace=True)

In [80]:
geojson_data = json.loads(rolling_map.to_json())

In [14]:
rolling_map['rolling_avg'].describe()

count    191.000000
mean       4.500639
std        2.147317
min        1.741177
25%        3.211796
50%        4.180022
75%        5.071962
max       20.874993
Name: rolling_avg, dtype: float64

In [92]:
min_val = rolling_map['rolling_avg'].min()
max_val = rolling_map['rolling_avg'].max()
norm = colors.Normalize(vmin=min_val, vmax=max_val)
cmap = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

In [84]:
classes = [0, 1, 2, 3, 4, 5, 7, 9,11,13, 15,17]
colorscale = ['#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026',
              '#660022', '#440016', '#33000F', '#20000A']
style = dict(weight=2, opacity=1, color='white', dashArray='3', fillOpacity=0.7)

In [85]:
ctg = ["{}+".format(cls, classes[i + 1]) for i, cls in enumerate(classes[:-1])] + ["{}+".format(classes[-1])]
colorbar = dlx.categorical_colorbar(categories=ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")

In [86]:
style_handle = assign("""function(feature, context){
    const {classes, colorscale, style, colorProp} = context.hideout;  // get props from hideout
    const value = feature.properties[colorProp];  // get value the determines the color
    for (let i = 0; i < classes.length; ++i) {
        if (value > classes[i]) {
            style.fillColor = colorscale[i];  // set the fill color according to the class
        }
    }
    return style;
}""")

In [18]:
def get_info(feature=None):
    header = [html.H4("On Time Rating")]
    if not feature:
        return header + [html.P("Hoover over a district")]
    return header + [html.B(feature["properties"]["NTAName"]), html.Br(),
                     "{} minutes off schedule".format(round(feature["properties"]["rolling_avg"],2))]

In [19]:
with open("shapefiles/segments.geojson", "r") as f:
    bus_geojson = json.load(f)

In [ ]:
app = Dash(__name__)
server = app.server

info = html.Div(children=get_info(), id="info", className="info",
                style={
        "position": "absolute",
        "top": "10px",
        "right": "10px",
        "zIndex": "1000",
        "background-color": "white", 
        "padding": "10px",  
        "border": "1px solid #ccc", 
        "border-radius": "5px",  
        "box-shadow": "0 0 5px rgba(0, 0, 0, 0.2)" 
    })



app.layout = html.Div([
    html.Div([
        html.H1("NYC Bus On Time Rating", style={'font-family': 'Georgia', 'padding': '10px', 'textAlign': 'center'}),
        html.P("This map shows a rolling average of on-time arrivals for each NTA in NYC. The on-time rating is calculated as the average number of minutes each bus is off schedule in either direction."),
        html.P("Hover over a district to see the name and on-time rating."),
    ], style={'font-family': 'Georgia', 'padding': '10px', 'textAlign': 'center'}),

    html.Div([
        dl.Map(center=[40.7128, -74.0060], zoom=10, children=[
            dl.TileLayer(),
            dl.GeoJSON(data=geojson_data, style=style_handle, id='geojson',
                       zoomToBounds=True, zoomToBoundsOnClick=True,
                       hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),
                       hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="rolling_avg")),
                        dl.GeoJSON(data = bus_geojson,
                       options=dict(style=dict(color="#00FF00", weight=3, opacity=0.9, dashArray="4"))),
            colorbar, info
        ], style={'width': '100%', 'height': '75vh', 'padding-bottom': '20px', 'margin-bottom': '50px'})
    ])
])


@app.callback(Output("info", "children"), Input("geojson", "hoverData"))
def info_hover(feature):
    return get_info(feature)

if __name__ == '__main__':
    app.run_server(port=8060,debug=True)

In [43]:
def make_table(df,month_dict):
    df['month_name'] = df['month'].map(month_dict)
    month = df['month_name'].unique()[0]
    boroughs = df.groupby('borough')['average_road_speed'].mean().reset_index(name='raw_speed')
    whole_city = df['average_road_speed'].mean()
    table = pd.DataFrame({'borough': 'NYC Whole', 'raw_speed': whole_city}, index=[0])
    boroughs = pd.concat([boroughs, table], ignore_index=True)
    boroughs[f'{month}_avg_speed'] = boroughs['raw_speed'].round(1)
    return boroughs

In [189]:
table_rows = table_data.to_dict('records')
table_columns = [{"name": col, "id": col} for col in table_data.columns]

In [34]:
oct_table = make_table(oct_speed,month_dict)
nov_table = make_table(nov_speed,month_dict)

In [41]:
def join_tables(df1,df2,month_dict):
    tb1 = make_table(df1,month_dict)
    tb2 = make_table(df2,month_dict)
    tables = tb1.merge(tb2,on='borough')
    drop_cols = [col for col in tables.columns if 'raw_speed' in col]
    tables.drop(columns=drop_cols,inplace=True)
    return tables

In [44]:
join_tables(oct_speed,nov_speed,month_dict)

,borough,October_avg_speed,November_avg_speed
0,Bronx,8.3,7.5
1,Brooklyn,6.8,6.8
2,Manhattan,5.8,5.6
3,Other,10.6,10.3
4,Queens,8.5,8.5
5,Staten Island,12.5,12.2
6,NYC Whole,8.2,8.0


In [90]:
app = Dash(__name__)
server = app.server

info = html.Div(children=get_info(), id="info", className="info",
                style={
        "position": "absolute",
        "top": "10px",
        "right": "10px",
        "zIndex": "1000",
        "background-color": "white", 
        "padding": "10px",  
        "border": "1px solid #ccc", 
        "border-radius": "5px",  
        "box-shadow": "0 0 5px rgba(0, 0, 0, 0.2)" 
    })

table_data = join_tables(oct_speed,nov_speed,month_dict)
table_rows = table_data.to_dict('records')
table_columns = [{"name": col, "id": col} for col in table_data.columns]

app.layout = html.Div([
    html.Div([
        html.H1("NYC Bus On Time Rating", style={'font-family': 'Georgia', 'padding': '10px', 'textAlign': 'center'}),
        html.P("The map below shows two metrics for bus performance in NYC. The color of the tiles represents a rolling average of on-time arrivals for each Neighborhood Tabulation Area (NTA) in NYC. The on-time rating is calculated as the average number of minutes each bus is off schedule in either direction. The green lines represent the slowest segments of bus routes in NYC. Hover over a district to see the name and on-time rating."),
        html.P("The on-time data is collected from the MTA's Bus Time API and is scraped everyday at rush hour. Bus speeds are collected from the NYC Open Data and are updated monthly. See table below map for average speeds by borough."),
        html.P(f"On-time data last scraped {scrape_date}"),
    ], style={'font-family': 'Georgia', 'padding': '10px', 'textAlign': 'center'}),

    html.Div([
        dl.Map(center=[40.7128, -74.0060], zoom=10, children=[
            dl.TileLayer(),
            dl.GeoJSON(data=geojson_data, style=style_handle, id='geojson',
                       zoomToBounds=True, zoomToBoundsOnClick=True,
                       hoverStyle=arrow_function(dict(weight=5, color='#666', dashArray='')),
                       hideout=dict(colorscale=colorscale, classes=classes, style=style, colorProp="rolling_avg")),
            dl.GeoJSON(data=bus_geojson,
                       options=dict(style=dict(color="#00FF00", weight=3, opacity=1))),
            colorbar, info
        ], style={'width': '100%', 'height': '75vh', 'padding-bottom': '20px', 'margin-bottom': '50px'})
    ]),
    html.Div([
        html.P("The table below shows the average speed of buses in each borough. The data is collected from the NYC Open Data and is updated monthly."),
    ])
    ,
    html.Div([
        html.H2("On-Time Rating Data", style={'font-family': 'Georgia', 'textAlign': 'center', 'margin-top': '20px'}),
        dash_table.DataTable(
            data=table_rows,
            columns=table_columns,
            style_table={'width': '80%', 'margin': '0 auto'},
            style_cell={'textAlign': 'center', 'font-family': 'Georgia', 'padding': '5px'},
            style_header={'fontWeight': 'bold', 'backgroundColor': '#f4f4f4'}
        )
    ])
])

@app.callback(Output("info", "children"), Input("geojson", "hoverData"))
def info_hover(feature):
    return get_info(feature)

if __name__ == '__main__':
    app.run_server(port=8060,debug=True)